In [1]:
# @FWS-IMPORTS add all import requirements here
from radlib_imports import *
from radlib.fw.flywheel_clients import uwhealth_client, uwhealthaz_client
from radlib.fws.fws_image import FWSImageFile, FWSImageType, FWSImageFileList
from radlib.fws.fws_utils import fws_input_file_list, fws_in_jupyter_notebook, fws_load_image
from radlib.dcm.utilities import get_sitk_dicom_tag
from radlib.projects.spore_17009_set import get_slice_conversion, get_pathology_file_list, update_dicom_tags_for_pathology_instance

In [3]:
fw_rad_client = uwhealthaz_client()
fw_path_client = uwhealth_client()

rad_group_id = 'prostatespore'
rad_project_label = 'DODProjectRead'
rad_subject_label = '17009_Pyl11'

path_group_id = 'prosspore'
path_project_label = 'Complete17009PathologyArchive'
path_subject_label = 'P11'

path_session_name = 'Pathology'
ref_session_name = 'Reference'

fws_files = fws_input_file_list(fw_rad_client, rad_project_label, subject_labels=[rad_subject_label], generate_code=False,
                                        local_root=None)

In [3]:
# FWS-INPUTS edit this cell to the flywheel file references that you need, add local_path(s) if necessary, then TURN OFF generate_code above
local_root = "None"
fws_input_files = FWSImageFileList( {
"PET1_Obl": FWSImageFile(fw_rad_client, fw_path="prostatespore/DODProjectRead/17009_Pyl11/PET1/Obl Axial T2 Prostate/Obl Axial T2 Prostate.dicom.zip",
	local_path="None"),
"PET1_PET": FWSImageFile(fw_rad_client, fw_path="prostatespore/DODProjectRead/17009_Pyl11/PET1/PET AC Prostate/PET AC Prostate.dicom.zip",
	local_path="None"),
})

In [7]:
# slice to pre and post mr conversions from excel file
pathology_slices, pre_slices, post_slices = get_slice_conversion(path_subject_label)

C:\Anaconda\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [8]:
rad_project = fw_rad_client.projects.find_one(f'label={rad_project_label}')
rad_subject = rad_project.subjects.find_one(f'label={rad_subject_label}')

In [9]:
# unfortunately we have three different cases to look for files, not worth reorganizing for now
if path_subject_label in ['P27', 'P28']:
    path_file_list = get_pathology_file_list(fw_path_client, path_group_id, path_project_label, path_subject_label, 'H_E Slices', 'Edited Scans')

elif path_subject_label in ['P29', 'P30']:
      path_file_list = get_pathology_file_list(fw_path_client, path_group_id, path_project_label, path_subject_label, 'Beebe Data', 'Tissue Data')

else:
    path_file_list = get_pathology_file_list(fw_path_client, path_group_id, path_project_label, path_subject_label, 'Beebe Data', 'H_E Slices')

In [10]:
obl_slices = fws_input_files['PET1_Obl'].load_image(image_type=FWSImageType.pydicom)

In [11]:
# for each pathology "slice", find the corresponding pre slice, open it, set the position of the pathology slice to it, and save as a dicom
path_images = []
path_used = []
new_path_series_instance_uid = pydicom.uid.generate_uid()

for path_slice_index, pre_slice_index, post_slice_index in zip(pathology_slices, pre_slices, post_slices):
    path_file_name = path_file_list[path_slice_index]
    if path_file_name is None:
        path_file_name = path_file_list[path_slice_index+1]
    if path_file_name in path_used or path_file_name is None:
        continue
        
    if path_subject_label == 'P27' or path_subject_label == 'P28':
        path_fw_path = f'{path_group_id}/{path_project_label}/{path_subject_label}/H_E Slices/Edited Scans/{path_file_name}'

    else:
        path_fw_path = f'{path_group_id}/{path_project_label}/{path_subject_label}/Beebe Data/H_E Slices/{path_file_name}'

    path_image = FWSImageFile(fw_path_client, fw_path=path_fw_path)

    path_image.load_image(image_type=FWSImageType.tif)
    path_slice = path_image.convert_image_to_dicom()
    path_slice_updated = update_dicom_tags_for_pathology_instance(path_subject_label, path_slice, path_slice_index, obl_slices[pre_slice_index], 
                                                                  new_path_series_instance_uid)

    path_images.append(path_slice_updated)
    path_used.append(path_file_name)

path_images = sorted(path_images, key=lambda s: s.GetOrigin()[2])

In [12]:
# save as dicom slices to zip file
# writer for dicom slices, keep UIDs
writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()

zip_root_dir = tempfile.gettempdir()
zip_folder = f'{zip_root_dir}/{path_session_name}'
if os.path.exists(zip_folder):
    shutil.rmtree(zip_folder)
os.mkdir(zip_folder)

for p, path_image in enumerate(path_images):
    writer.SetFileName(f'{zip_folder}/{rad_subject_label}_pathology_{(p+1):02}.dcm')
    writer.Execute(path_image)
from zipfile import ZipFile
import glob
zip_path = f'{zip_root_dir}\\pathology_images.dicom.zip'
with ZipFile(zip_path, 'w') as zip_file:
    for file_path in glob.glob(f'{zip_folder}/*.dcm'):
        zip_file.write(file_path, f'/{os.path.basename(file_path)}')

In [14]:
# upload the result to flywheel
ack = rad_subject.sessions.find(f'label={path_session_name}')[0].add_acquisition(label='pathology_images4')
ack.upload_file(zip_path)

[{'classification': {},
  'client_hash': None,
  'copy_of': None,
  'created': datetime.datetime(2025, 3, 27, 18, 12, 54, 98832, tzinfo=tzutc()),
  'deid_log_id': None,
  'deid_log_skip_reason': None,
  'deleted': None,
  'file_id': '67e595263b5f26f5cd8741aa',
  'gear_info': None,
  'hash': 'f77202b83a9ba3aef9beba222ead15629fbdaa5311b052cd56563322481f34622aa00071fe1a61cb619799c2b5ee5816',
  'id': '5604935c-5da9-4aee-8515-3b349408fb3b',
  'info': {},
  'info_exists': False,
  'mimetype': 'application/zip',
  'modality': None,
  'modified': datetime.datetime(2025, 3, 27, 18, 12, 54, 98832, tzinfo=tzutc()),
  'name': 'pathology_images.dicom.zip',
  'origin': {'id': 'ckashuk@wisc.edu', 'type': <OriginType.USER: 'user'>},
  'original_copy_of': None,
  'parent_ref': {'id': '67e59522c3d6cef71ca37cea', 'type': 'acquisition'},
  'parents': {'acquisition': '67e59522c3d6cef71ca37cea',
              'analysis': None,
              'group': 'prostatespore',
              'project': '67c0a7636cd3bf1

In [ ]:
# make regridded pet dicom slices like obl
pet_regrid = sitk.GetArrayFromImage(sitk.ReadImage(pet_regrid_nii_gz_path))
pet_regrid = pet_regrid.swapaxes(0, 2)
pet_regrid = pet_regrid.swapaxes(0, 1)

new_series_instance_uid = pydicom.uid.generate_uid()

for p, obl_slice in enumerate(obl_slices):
    pet_slice_img = pet_regrid[:, :, p]
    pet_slice_img[pet_slice_img > 32000] = 32000
    pet_slice_img = [np.astype(pet_regrid[:, :, p], np.int16)]
    pet_slice = sitk.GetImageFromArray(pet_slice_img)
    pet_slice.CopyInformation(obl_slice)
    pet_slice = update_dicom_tags_for_mr_instance(pet_slice, obl_slice, series_id=new_series_instance_uid)
    pet_slice.SetMetaData('0008|0060', "PT")
    pet_slice.SetMetaData('0008|103e', f'{rad_subject_label} regridded PET')

    writer.SetFileName(f'{zip_folder}/{rad_subject_label}_pet_regrid_{(p + 1):02}.dcm')
    writer.Execute(pet_slice)

In [ ]:
# save as dicom slices to zip file
zip_root_dir = tempfile.gettempdir()
zip_folder = f'{zip_root_dir}/PET_regrid'
if os.path.exists(zip_folder):
    shutil.rmtree(zip_folder)
os.mkdir(zip_folder)

zip_path = f'{zip_folder}/PET_regrid.dicom.zip'
with ZipFile(zip_path, 'w') as zip_file:
    for file_path in glob.glob(f'{zip_folder}/*.dcm'):
        zip_file.write(file_path, f'/{os.path.basename(file_path)}')

# upload to flywheel
ack = rad_subject.sessions.find(f'label={path_session_name}')[0].add_acquisition(label='fws_PET_regrid')
ack.upload_file(zip_path)